In [53]:
import os
import json
import pandas as pd

from sqlalchemy import create_engine, text

In [2]:
file_path = '/Volumes/Expansion/데이터분석 프로젝트/google_analytics_customer/train.csv'
df = pd.read_csv(file_path, low_memory=False)

df.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [ ]:
def load_df(csv_path):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path,
                     converters={column: json.loads for column in JSON_COLUMNS},
                     dtype={'fullVisitorId': 'str'} # Important!!
                    )

    for column in JSON_COLUMNS:
        column_as_df = pd.json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [30]:
df = load_df('/Volumes/Expansion/데이터분석 프로젝트/google_analytics_customer/train.csv')

Loaded train.csv. Shape: (903653, 55)


In [31]:
username = 'root'
password = 'star&0831'
host = 'localhost'
port = '3306'
database = 'ecommerce'

# 엔진 생성
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [32]:
df.to_sql(name='ga_train', con=engine, index=False, if_exists='replace')

903653

In [33]:
result = pd.read_sql(
    "SELECT * FROM ga_train LIMIT 5;",
    con=engine
)

result

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,None,None,None,None,None,None,None,None
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,None,None,None,None,None,None,None,None
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,None,None,None,None,None,None,None,None
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,not available in demo dataset,NaN,None,None,None,None,None,None,None,None
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,not available in demo dataset,1.0,None,None,None,None,None,None,None,None


In [43]:
query = """
    DESC ga_train
"""

desc_df = pd.read_sql(query, con=engine)

desc_df

,Field,Type,Null,Key,Default,Extra
0,channelGrouping,text,YES,,None,
1,date,bigint,YES,,None,
2,fullVisitorId,text,YES,,None,
3,sessionId,text,YES,,None,
4,socialEngagementType,text,YES,,None,
5,visitId,bigint,YES,,None,
6,visitNumber,bigint,YES,,None,
7,visitStartTime,bigint,YES,,None,
8,device.browser,text,YES,,None,
9,device.browserVersion,text,YES,,None,


- 지속적으로 분석할 totals.transactionRevenue 컬럼이 text형
- 이를 bigint로 형변환 (bigint로 형변환된 컬럼을 새로 생성)

In [56]:
with engine.begin() as conn :
    # conn.execute(text("""
    #     ALTER TABLE ga_train
    #     ADD COLUMN revenue_int BIGINT;
    # """))
    
    conn.execute(text("""
        UPDATE ga_train
        SET revenue_int = CAST(`totals.transactionRevenue` AS UNSIGNED);
    """))

    conn.execute(text("""
        UPDATE ga_train
        SET revenue_int = 0
        WHERE revenue_int IS NULL;
    """))

In [60]:
query = """
    SHOW columns
    FROM ga_train LIKE 'revenue_int';
"""

result = pd.read_sql(query, con=engine)
result

,Field,Type,Null,Key,Default,Extra
0,revenue_int,bigint,YES,,None,


## 수익이 있는 고객만 필터링하고 요약 분석

In [61]:
query = """
    SELECT channelGrouping,
           date,
           fullVisitorId,
           revenue_int AS revenue
    FROM ga_train
    WHERE revenue_int > 0
"""

df_result = pd.read_sql(query, con=engine)

df_result

,channelGrouping,date,fullVisitorId,revenue
0,Direct,20160902,6194193421514403509,37860000
1,Organic Search,20160902,5327166854580374902,306670000
2,Referral,20160902,8885051388942907862,68030000
3,Referral,20160902,0185467632009737931,26250000
4,Referral,20160902,3244885836845029978,574150000
...,...,...,...,...
11510,Referral,20170104,6849604863010168468,335260000
11511,Referral,20170104,0211986650009953276,599950000
11512,Referral,20170104,9981562221973608723,30750000
11513,Direct,20170104,963126344221316701,69390000


## 수익이 있는 고객 대상 유입 채널(channelGrouping)별 총 방문 수, 구매 전환 수, 전환율(%)

In [63]:
query = """
    SELECT channelGrouping,
           COUNT(*) AS visit_num,
           COUNT(CASE WHEN revenue_int > 0 THEN 1 END) AS conversion,
           ROUND(
            COUNT(CASE WHEN revenue_int > 0 THEN 1 END) / COUNT(*), 2) * 100 AS conversion_rate
    FROM ga_train
    GROUP BY channelGrouping
    ORDER BY conversion_rate DESC;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,channelGrouping,visit_num,conversion,conversion_rate
0,Referral,104838,5311,5.0
1,Paid Search,25326,468,2.0
2,Display,6262,142,2.0
3,Organic Search,381561,3438,1.0
4,Direct,143026,2042,1.0
5,(Other),120,1,1.0
6,Affiliates,16403,9,0.0
7,Social,226117,104,0.0


## 유입 브라우저별 수익 기여도 분석

In [89]:
query = """
    SELECT `device.browser` browser,
           ROUND(AVG(revenue_int), 2) avg_revenue,
           SUM(revenue_int) total_revenue,
           COUNT(CASE WHEN revenue_int > 0 THEN 1 END) conversion_num
    FROM ga_train
    GROUP BY browser
    ORDER BY conversion_num DESC
    LIMIT 10;
"""

df_result = pd.read_sql(query, con=engine)

df_result

,browser,avg_revenue,total_revenue,conversion_num
0,Chrome,2229505.52,1.383105e+12,10353
1,Safari,287511.21,5.239748e+10,780
2,Firefox,2409461.27,8.931632e+10,191
3,Internet Explorer,426092.90,8.255550e+09,109
4,Edge,635273.89,6.482970e+09,58
5,Safari (in-app),22448.18,1.537700e+08,12
6,Android Webview,14602.67,1.148500e+08,6
7,Opera,38162.33,2.153500e+08,5
8,Amazon Silk,53458.11,2.999000e+07,1
9,BlackBerry,0.00,0.000000e+00,0


## 디바이스 카테고리별 수익 분석

In [ ]:
query = """
    SELECT `device.deviceCategory` device,
           ROUND(AVG(revenue_int), 2) avg_revenue,
           SUM(revenue_int) total_revenue,
           COUNT(CASE WHEN revenue_int > 0 THEN 1 END) conversion_num
    FROM ga_train
    GROUP BY device
    ORDER BY conversion_num DESC;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,device,avg_revenue,total_revenue,conversion_num
0,desktop,2228609.32,1.480864e+12,10495
1,mobile,238523.46,4.978581e+10,852
2,tablet,309413.77,9.421340e+09,168


## 방문 횟수별 수익 변화 분석

In [102]:
query = """
    SELECT visitNumber,
            ROUND(AVG(revenue_int) / 1e6, 2) avg_revenue
    FROM ga_train
    GROUP BY visitNumber
    ORDER BY avg_revenue DESC;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,visitNumber,avg_revenue
0,243,5782.38
1,237,2892.61
2,305,2332.56
3,181,1785.55
4,258,1403.61
...,...,...
379,376,0.00
380,379,0.00
381,378,0.00
382,377,0.00


## VIP 고객(수익 상위 20% 고객) 확인

In [ ]:
query = """
    SELECT fullVisitorId,
           revenue_int
    FROM (
        SELECT *,
               NTILE(5) OVER (ORDER BY revenue_int DESC) revenue_section
        FROM ga_train
        WHERE revenue_int > 0
        ORDER BY revenue_section DESC
    ) revenue_rank
    WHERE revenue_section = 1
    ORDER BY revenue_int DESC;
"""

df_result = pd.read_sql(query, con=engine)
df_result


,fullVisitorId,revenue_int
0,1957458976293878100,23129500000
1,1957458976293878100,17855500000
2,5632276788326171571,16023750000
3,9417857471295131045,10589140000
4,1957458976293878100,8677830000
...,...,...
2298,0355692456617973868,137540000
2299,0162341046495604782,137330000
2300,5384999715597179184,137290000
2301,013937734216012052,137270000


## VIP 고객 대상 디바이스별 고객 수

In [112]:
query = """
    SELECT `device.deviceCategory` device,
           COUNT(*) VIP_num
    FROM (
        SELECT *,
               NTILE(5) OVER (ORDER BY revenue_int DESC) revenue_section
        FROM ga_train
        WHERE revenue_int > 0
    ) revenue_rank
    WHERE revenue_section = 1
    GROUP BY device
    ORDER BY VIP_num DESC;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,device,VIP_num
0,desktop,2242
1,mobile,48
2,tablet,13


## 가장 많은 VIP 고객을 보유한 국가

In [114]:
query = """
    SELECT `geoNetwork.country` country,
           COUNT(*) VIP_num
    FROM (
        SELECT *,
               NTILE(5) OVER (ORDER BY revenue_int DESC) revenue_section
        FROM ga_train
        WHERE revenue_int > 0
    ) revenue_rank
    WHERE revenue_section = 1
    GROUP BY country
    ORDER BY VIP_num DESC
    LIMIT 10;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,country,VIP_num
0,United States,2176
1,Canada,39
2,Venezuela,25
3,Indonesia,5
4,Japan,4
5,Hong Kong,4
6,Mexico,4
7,United Kingdom,3
8,Australia,3
9,Puerto Rico,3


## 날짜별 방문 사용자 수

In [124]:
query = """
    SELECT DATE_FORMAT(date, '%%Y-%%m-%%d') date,
           COUNT(DISTINCT fullVisitorId) visit_num
    FROM ga_train
    GROUP BY date
    ORDER BY date;
"""

df_result = pd.read_sql(query, con=engine)
df_result

,date,visit_num
0,2016-08-01,1569
1,2016-08-02,1961
2,2016-08-03,2657
3,2016-08-04,2947
4,2016-08-05,2492
...,...,...
361,2017-07-28,2233
362,2017-07-29,1498
363,2017-07-30,1649
364,2017-07-31,2368


## 시간대별, 요일별 총 수익

In [163]:
# 방문 시간대별
query = """
    SELECT DATE_FORMAT(datetime_utc, '%%H') hour,
           ROUND(SUM(revenue_int) / 1e6, 2) total_revenue
    FROM (
        SELECT *,
               FROM_UNIXTIME(visitStartTime) datetime_utc
        FROM ga_train
    ) visitTime
    GROUP BY hour
    ORDER BY hour
"""

df_result = pd.read_sql(query, con=engine)
print(df_result)

   hour  total_revenue
0    00      122407.92
1    01      102336.10
2    02      131075.50
3    03      147468.64
4    04      131365.99
5    05      150983.42
6    06      104293.41
7    07       89913.38
8    08       66599.38
9    09       60611.86
10   10       55662.53
11   11       50839.91
12   12       36967.37
13   13       34342.49
14   14       24812.97
15   15       13246.10
16   16       10748.61
17   17        9655.55
18   18        5866.27
19   19        7409.81
20   20       10048.42
21   21       21489.86
22   22       62449.34
23   23       89476.41


In [161]:
# 방문 요일별
# 방문 시간대별
query = """
    SELECT DATE_FORMAT(date, '%%W') day_of_week,
           ROUND(SUM(revenue_int) / 1e6, 2) total_revenue
    FROM ga_train
    GROUP BY day_of_week
    ORDER BY CASE
                WHEN day_of_week = 'Monday' THEN 1
                WHEN day_of_week = 'Tuesday' THEN 2
                WHEN day_of_week = 'Wednesday' THEN 3
                WHEN day_of_week = 'Thursday' THEN 4
                WHEN day_of_week = 'Friday' THEN 5
                WHEN day_of_week = 'Saturday' THEN 6
                WHEN day_of_week = 'Sunday' THEN 7
            END
"""

df_result = pd.read_sql(query, con=engine)
print(df_result)

  day_of_week  total_revenue
0      Monday      272050.26
1     Tuesday      307466.24
2   Wednesday      298488.44
3    Thursday      269170.56
4      Friday      258656.66
5    Saturday       58397.36
6      Sunday       75841.72
